## Employee Scheduling Solver

This notebook was created to help solve employee scheduling problem during post-peak Coronavirus period.

In my company, during Coronavirus lockdown, most people in my part of the organization worked from home. When the virus spread subsided, we started preparing for a controlled return to the office. One of many safety measures we implemented was limiting the number of workspots per floor to maintain 1.5 meter distance.

With the number of workspots reduced, not everyone can come to the office at the same time. Everyone has a preference on which days to come in. Teams that work together should come together. Figuring out a schedule manually, even for a small group of people, is a tedious and error-prone task. There must be a better way...

One way of looking at schedule construction is treating it like a mathematical optimization problem:
* Define "utility" value of a schedule as a sum of happiness of employees. Coming in on a preferred day makes an employee happy. Coming in on a non-preferred day makes an employee very unhappy. Coming in together with teammates makes employees happier. 
* Maximize the "utility" of a schedule, given constraints:
 - Number of employees who come in on any given day should not exceed the number of available workspots.
 - Optional: number of times an employee comes in each week should not exceed an arbitrary number, to give everyone a chance to come to the office.
 
This type of optimization problem can be solved by a Mixed Integer Linear Program (MILP) solver. The code below shows how to construct such a problem, feed it to the solver, and show the solution.

_Note: this is my first program in Julia, feedback on making it better is welcome!_

In [7]:
using JuMP
using Cbc
using Combinatorics
using Base.Iterators
using XLSX

### Utility function - toy data problem

In [8]:
function load_toy_data()
    employee_names = ["Tony", "Bruce", "Steve", "Natasha", "Wanda", "Susan"]
    day_names = ["1.1", "1.2", "1.3", "1.4", "1.5", "2.1", "2.2", "2.3", "2.4", "2.5",]

    # Rows: employees, columns: days
    # 5: Strong preference to come in (happy)
    # 1: Weak preference to come in (happy-ish)
    # -99: Unavailable to come in (very unhappy)

    preferences = [
          5   1   5   5   5   5   1   5   5   5
          1   5   1   1 -99   1   5   1   1 -99
          1   5   1   5   1   1 -99   1   5   5
          1   1 -99   1   1   1   1 -99   1   1
          5   5   5 -99   1   1   5   5   5   1
          1 -99   1   5   5   5   1   1   1   1
    ]

    # Connection matrix - upper triangular matrix
    # Item at row i column j shows strength of connection between employees i and j.
    # #he higher the number, the closer the collaboration.
    # 
    # Only the upper right triangle is useful. The diagnoal represents self-connection (which is useless for our purpose)
    # and the bottom left is a mirror of upper right (but we don't have to fill it in).
    # The algorithm only uses the upper right triangle and ignores the rest.

    connections = [
        0 8 8 1 1 1
        0 0 8 1 1 1
        0 0 0 1 1 1
        0 0 0 0 1 1
        0 0 0 0 0 1
        0 0 0 0 0 0
    ];
    return (employee_names, day_names, preferences, connections)
end

load_toy_data (generic function with 1 method)

### Utility function - load data from Excel sheet

In [9]:
# The data is loaded from named ranges. See example sheet for reference.
# 
# Preferences of employees to attend each day are indicated with
#   Y (yes, strength 5), M (maybe, strength 1) and N (no, strength -99).
# Empty cells are treated as "N"
#
# Connections between employees are indicated by integers. Positive values will steer the scheduler
# to schedule those people together. Negative values can be used to ensure that some people are never scheduled together.
# Empty cells are treated as 0.

function load_data(input_file_name)
    xf = XLSX.openxlsx(input_file_name)
    sheet = xf[1]
    my_employee_names = sheet["EmployeeNames"]
    my_day_names = sheet["DayNames"]
    my_preferences = sheet["Preferences"]
    my_connections = sheet["Connections"]
    close(xf)

    day_names = string.(reshape(my_day_names, size(my_day_names)[2]) )
    employee_names = string.(reshape(my_employee_names, size(my_employee_names)[1]))
    connections = Int64.(replace(my_connections, missing => 0))
    preferences = Int64.(replace(lowercase.(replace(my_preferences, missing => "N")), "y" => 5, "m" => 1, "n" => -99))

    n_days = size(day_names)[1]
    n_employees = size(employee_names)[1]
    @assert (n_employees, n_employees) == size(connections)
    @assert (n_employees, n_days) == size(preferences)
    return (employee_names, day_names, preferences, connections)
end

load_data (generic function with 1 method)

#### Build a toy problem, or load real data from Excel

In [19]:
NUM_SPOTS = 2
MAX_VISITS_PER_WEEK = 2
employee_names, day_names, preferences, connections = load_toy_data()

# input_file_name = "input.xlsx"
# NUM_SPOTS = 72 # Change this to the number of workspots available
# MAX_VISITS_PER_WEEK = 5
# employee_names, day_names, preferences, connections = load_data(input_file_name)

(["Tony", "Bruce", "Steve", "Natasha", "Wanda", "Susan"], ["1.1", "1.2", "1.3", "1.4", "1.5", "2.1", "2.2", "2.3", "2.4", "2.5"], [5 1 … 5 5; 1 5 … 1 -99; … ; 5 5 … 5 1; 1 -99 … 1 1], [0 8 … 1 1; 0 0 … 1 1; … ; 0 0 … 0 1; 0 0 … 0 0])

### Compute utility variables

In [11]:
function get_optimization_ranges(employee_names, day_names)
    employees = 1:length(employee_names)
    days = 1:length(day_names)
    return (employees, days)
end

employees, days = get_optimization_ranges(employee_names, day_names)
employee_pairs = collect(combinations(employees, 2));

### Construct a linear model

In [12]:
function build_model(employee_names::Array{String,1}, day_names::Array{String,1},
        preferences::Array{Int64,2}, connections::Array{Int64,2}, 
        num_spots::Int64, max_visits_per_week::Int64)

    roster_model = Model(Cbc.Optimizer)

    employees, days = get_optimization_ranges(employee_names, day_names)
    employee_pairs = collect(combinations(employees, 2));

    # Attendance variables - employee per day
    attendance_vars = Dict((e, d) => @variable(roster_model, base_name="atnde_$(e)d_$(d)", binary=true) 
        for e in employees for d in days)

    # Linearization variables
    z_vars = Dict((e1, e2, d) => @variable(roster_model, base_name="ze_$(e1)e_$(e2)d_$(d)", binary=true)
        for (e1, e2) in employee_pairs for d in days)

    # Maximize the sum of preferences of all attending employees and the sum of connections between present employees
    @objective(roster_model, Max, 
        sum(preferences[e, d] * attendance_vars[(e, d)] for e in employees for d in days) + 
        sum(z_vars[(e1, e2, d)] * connections[e1, e2] for (e1, e2) in employee_pairs for d in days))

    # Limit the number of employees attending each day to NUM_SPOTS
    for d in days
        @constraint(roster_model, sum(attendance_vars[(e, d)] for e in employees) <= num_spots)
    end

    # Limit max number of visits per employee per week to MAX_VISITS_PER_WEEK
    for week in partition(days, 5)
        for e in employees
            @constraint(roster_model, sum(attendance_vars[(e, d)] for d in week) <= max_visits_per_week)
        end
    end

    # Linearization constraints
    for d in days
        for (e1, e2) in employee_pairs
            @constraint(roster_model, z_vars[(e1, e2, d)] <= attendance_vars[(e1, d)])
            @constraint(roster_model, z_vars[(e1, e2, d)] <= attendance_vars[(e2, d)])
            @constraint(roster_model, z_vars[(e1, e2, d)] >= attendance_vars[(e1, d)] + attendance_vars[(e2, d)] - 1)
        end
    end
    return roster_model, attendance_vars
end

model, attendance_vars = build_model(employee_names, day_names, preferences, connections, NUM_SPOTS, MAX_VISITS_PER_WEEK)

(A JuMP Model
Maximization problem with:
Variables: 210
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 150 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 322 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 210 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc), Dict{Tuple{Int64,Int64},VariableRef}((3, 6) => atnde_3d_6,(6, 9) => atnde_6d_9,(4, 4) => atnde_4d_4,(3, 1) => atnde_3d_1,(1, 10) => atnde_1d_10,(4, 5) => atnde_4d_5,(2, 4) => atnde_2d_4,(6, 5) => atnde_6d_5,(4, 9) => atnde_4d_9,(1, 2) => atnde_1d_2…))

### Run optimization

In [13]:
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 172.45 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 415 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 436 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 397 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 298 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 220 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 207 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 44 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 18 strengthened rows, 0 substitutions
Cgl0004I processed model has 232 rows, 210 columns (210 integer (210 of which binary)) and 1830 elements
Cbc0012I Integer solution of -125 found by DiveCoefficient after 0 iterations and 0 nodes

#### Print results - watch out, can be large, depending on the size of your problem.

In [15]:
println("Status = ", termination_status(model))
println("Solution = ", objective_value(model))
println()

for e in employees
    print("Employee $(e): ")
    for d in days
        print(convert(Int8, value(attendance_vars[(e, d)])), ", ")
    end
    println()
end

Status = OPTIMAL
Solution = 125.0

Employee 1: 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 
Employee 2: 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 
Employee 3: 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 
Employee 4: 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 
Employee 5: 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 
Employee 6: 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 


### Utility funciton - save solution to an Excel file

In [16]:
function save_solution(output_file_name::String, 
        employee_names::Array{String,1}, 
        day_names::Array{String,1}, attendance_vars)

    employees, days = get_optimization_ranges(employee_names, day_names)
    allocations = [[convert(Int64, value(attendance_vars[(e, d)])) for d in days] for e in employees]

    XLSX.openxlsx(output_file_name, mode="w") do xf
        sheet = xf[1]
        XLSX.rename!(sheet, "schedule")
        sheet[2, 1, dim=1] = employee_names # dim=1 means column
        sheet[1, 2, dim=2] = day_names # dim=2 means row
        for e in employees
            for d in days
                sheet[1 + e, 1 + d] = allocations[e][d]
            end
        end
    end
end

save_solution (generic function with 1 method)

In [18]:
output_file_name = "schedule.xlsx"
save_solution(output_file_name, employee_names, day_names, attendance_vars)